In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock
from model.model import Img2MathModel


In [6]:
img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=16, device=torch.device('mps'))
img, label = next(iter(data.train))
vocab_size = len(data.tokenizer.get_vocab())
enc_output = torch.randn(1, 512, 512)
model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch=9-step=65180.ckpt')
model.eval()
model.generate(img[0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4 img, label = next(iter(data.train))                                                          │
│   5 vocab_size = len(data.tokenizer.get_vocab())                                                 │
│   6 enc_output = torch.randn(1, 512, 512)                                                        │
│ ❱ 7 model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch     │
│   8 model.generate(img[0])                                                                       │
│   9                                                                                              │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/core/module │
│ .py:1520 in load_from_checkpoint                                                                 │
│                                                                                                  │
│   1517 │   │   │   pretrained_model.freeze()                                                     │
│   1518 │   │   │   y_hat = pretrained_model(x)                                                   │
│   1519 │   │   """                                                                               │
│ ❱ 1520 │   │   loaded = _load_from_checkpoint(                                                   │
│   1521 │   │   │   cls,                                                                          │
│   1522 │   │   │   checkpoint_path,                                                              │
│   1523 │   │   │   map_location,                                                                 │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/core/saving │
│ .py:61 in _load_from_checkpoint                                                                  │
│                                                                                                  │
│    58 │   **kwargs: Any,                                                                         │
│    59 ) -> Union["pl.LightningModule", "pl.LightningDataModule"]:                                │
│    60 │   with pl_legacy_patch():                                                                │
│ ❱  61 │   │   checkpoint = pl_load(checkpoint_path, map_location=map_location)                   │
│    62 │                                                                                          │
│    63 │   # convert legacy checkpoints to the new format                                         │
│    64 │   checkpoint = _pl_migrate_checkpoint(                                                   │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/fabric/utilities/cl │
│ oud_io.py:51 in _load                                                                            │
│                                                                                                  │
│   48 │   │   )                                                                                   │
│   49 │   fs = get_filesystem(path_or_url)                                                        │
│   50 │   with fs.open(path_or_url, "rb") as f:                                                   │
│ ❱ 51 │   │   return torch.load(f, map_location=map_location)  # type: ignore[arg-type]           │
│   52                                                       

In [12]:
label['attention_mask'].shape

torch.Size([16, 256])

In [13]:
batch = next(iter(data.train))
img, labels = batch
trg_seq, input_mask = labels['input_ids'], labels['attention_mask']
zero = torch.zeros(1, 1)
input_seq = torch.cat((zero, trg_seq), dim=-1)
input_seq = input_seq[:, :-1].int()
input_mask = torch.cat((zero, input_mask), dim=-1)
input_mask= input_mask[:, :-1].int()
model(img[0], input_seq=input_seq, trg_seq=trg_seq, mask=input_mask)

(tensor([[[ 0.1854, -1.0171,  1.1940,  ..., -0.5155, -0.2047, -0.7936],
          [-0.1163,  0.2031,  1.0163,  ..., -0.7860,  0.8499, -0.9828],
          [-0.1543,  0.6852,  0.7890,  ...,  1.0069,  1.0043, -0.1216],
          ...,
          [ 0.4720, -0.0263,  0.6055,  ...,  0.2206, -0.2633, -0.3533],
          [-0.2364, -0.2472,  0.4905,  ...,  0.2950,  0.4586, -0.2331],
          [-0.2259,  1.5125,  1.0990,  ...,  0.0903,  0.7188, -0.4231]]],
        grad_fn=<ViewBackward0>),
 tensor(7.9361, grad_fn=<NllLossBackward0>))

In [6]:
logger = WandbLogger(project='img2math')

trainer = L.Trainer(limit_train_batches=10000, max_epochs=1, log_every_n_steps=20, deterministic=True,
                    logger=logger, accelerator='mps')

trainer.fit(model, data.train, data.test)
wandb.finish()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | ViT     | 8.9 M 
1 | decoder | Decoder | 14.9 M
------------------------------------
23.9 M    Trainable params
0         Non-trainable params
23.9 M    Total params
95.443    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss,▅▁▄▃▆▇▅▆▆▅▄▅▃▃▂▂▂▅▄▅▂▃▄▇▃▆▅▃█▃▇▄▁▄▇▅▅█▅▆
epoch,0
trainer/global_step,5039
val/loss,1.25875


torch.Size([1])